In [ ]:
import pandas as pd
import seaborn as sns
from datetime import datetime

from sqlalchemy import *
import json

import os
from dotenv import load_dotenv
load_dotenv()

## Load owned games into MySQL

In [ ]:
dict_owned_games = {}
with open('data/user_data.txt', 'r') as f:
    for raw_string in f.readlines(): 
        user_id, lst_inventory = list(json.loads(raw_string).items())[0]
        if lst_inventory:
            for i in lst_inventory:
                app_id = i.get('appid')
                playtime_forever = i.get('playtime_forever', 0)
                if playtime_forever > 0:
                    dict_owned_games.update({
                        user_id : {
                            'app_id' : app_id,
                            'playtime_forever' : playtime_forever
                        }
                    })

In [ ]:
df_owned_games = pd.DataFrame.from_dict(dict_owned_games, orient='index')
df_owned_games = df_owned_games.reset_index().rename({'index':'user_id'}, axis=1)
df_owned_games

In [ ]:
df_owned_games.to_sql(
        'steam_owned_games', 
        engine, 
        if_exists='replace', 
        index=False, 
        dtype={
            'user_id': BigInteger(),
            'app_id': Integer(),
            'playtime_forever': Integer()
        }, 
        chunksize = 10000
    )

## Clean app details

In [ ]:
def parse_steam_app_details(app_data):
    app_type = app_data.get('type')
    name = app_data.get('name')
    required_age = app_data.get('required_age')
    if app_data.get('is_free') == True:
        initial_price = 0
        currency = 'USD'
    else:
        if app_data.get('price_overview',{}):
            initial_price = app_data.get('price_overview',{}).get('initial', 0) / 100
            currency = app_data.get('price_overview',{}).get('currency')
        else:
            initial_price = None
            currency = None
    short_description = app_data.get('short_description')
    if not short_description:
        short_description = None
    header_image = app_data.get('header_image')
    supported_languages = app_data.get('supported_languages')
    if supported_languages:
        supported_languages = supported_languages.replace('<strong>*</strong>', '').replace('<br>languages with full audio support','')   
    developers = ', '.join(app_data.get('developers', []))
    if not developers:
        developers = None
    publishers = ', '.join(app_data.get('publishers', []))
    if not publishers:
        publishers = None
    fullgame = app_data.get('fullgame',{}).get('appid')
    lst_categories = app_data.get('categories',[])
    if lst_categories:
        categories = ', '.join([i.get('description') for i in lst_categories])
    else:
        categories = None
    lst_genres = app_data.get('genres',[])
    if lst_genres:
        genres = ', '.join([i.get('description') for i in lst_genres])
    else:
        genres = None
    if app_data.get('release_date',{}).get('coming_soon') == False:
        release_date = app_data.get('release_date',{}).get('date')
        #print(release_date)
        if release_date:
            try:
                release_date = datetime.strptime(release_date, '%b %d, %Y').date()
            except Exception as e:
                try:
                    release_date = datetime.strptime(release_date, '%d %b, %Y').date()
                except:
                    try:
                        release_date = datetime.strptime(release_date, '%b %Y').date()
                    except:
                        release_date = None
        else:
            release_date = None
    else:
        release_date = None

    dic_steam_app = {
        'name' : name,
        'type' : app_type,
        'release_date' : release_date,
        'currency' : currency,
        'initial_price' : initial_price,
        'short_description' : short_description,
        'header_image' : header_image,
        'fullgame' : fullgame,
        'developers' : developers,
        'publishers' : publishers,
        'required_age' : required_age,
        'supported_languages' : supported_languages,
        'categories' : categories,
        'genres' : genres,
    }

    return dic_steam_app

In [ ]:
dic_app_details = {}
with open('data/app_detail.txt', 'r') as f:
    for i in f.readlines():
        # catch empty lines
        try:
            for app_id, dic_response in json.loads(i).items():
                if dic_response.get('success'):
                    dic_app_details[app_id] = parse_steam_app_details(dic_response.get('data',{}))
        except:
            pass

In [ ]:
df_steam_app = pd.DataFrame.from_dict(dic_app_details, 'index')
df_steam_app.index.name = 'app_id'
df_steam_app.reset_index(inplace=True)
df_steam_app.head()

## Load app details into MySQL

In [ ]:
user = os.getenv('DB_USER')
password = os.getenv('DB_PASS')
host = os.getenv('DB_HOST')
db_name = os.getenv('DATABASE')
engine = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{db_name}?charset=utf8mb4')

In [ ]:
df_steam_app.to_sql(
    'steam_app_details', 
    engine, 
    if_exists='replace', 
    index=False, 
    chunksize = 10000, 
    dtype={'app_id':Integer(), 'required_age':Integer()})